In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from utils.models import create_vae_model
from utils.losses import reconstruction_loss
from utils.callbacks import SaveDecoderOutput, SaveDecoderModel

In [3]:
physical_gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_gpus[0], True)

In [4]:
def parse_fn(dataset, input_size=(28, 28)):
    x = tf.cast(dataset['image'], tf.float32)
    x = tf.image.resize(x, input_size)
    x = x / 255.
    return x, x

In [21]:
dataset = 'fashion_mnist'     # 'cifar10', 'fashion_mnist', 'mnist'
log_dirs = 'logs/fashion_mnist'
batch_size = 16
latent_dim = 2
input_shape = (28, 28, 1)

In [22]:
# Load datasets
train_data = tfds.load(dataset, split=tfds.Split.TRAIN)
test_data = tfds.load(dataset, split=tfds.Split.TEST)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\fashion_mnist\3.0.1.incompleteS5MNWG\fashion_mnist-train.tfrecord*...:   0%|  …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\fashion_mnist\3.0.1.incompleteS5MNWG\fashion_mnist-test.tfrecord*...:   0%|   …

Dataset fashion_mnist downloaded and prepared to ~\tensorflow_datasets\fashion_mnist\3.0.1. Subsequent calls will reuse this data.


In [23]:
# Setting datasets
AUTOTUNE = tf.data.experimental.AUTOTUNE  # 自動調整模式
train_data = train_data.shuffle(1000)
train_data = train_data.map(parse_fn, num_parallel_calls=AUTOTUNE)
train_data = train_data.batch(batch_size)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)
test_data = test_data.map(parse_fn, num_parallel_calls=AUTOTUNE)
test_data = test_data.batch(batch_size)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

In [24]:
# Callbacks function
model_dir = log_dirs + '/models'
os.makedirs(model_dir, exist_ok=True)
model_tb = keras.callbacks.TensorBoard(log_dir=log_dirs)
model_sdw = SaveDecoderModel(model_dir + '/best_model.h5', monitor='val_loss')
model_testd = SaveDecoderOutput(28, log_dir=log_dirs)

In [25]:
# create vae model
vae_model = create_vae_model(input_shape, latent_dim)

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 28, 28, 32)   320         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 14, 14, 64)   18496       conv2d_5[0][0]                   
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 7, 7, 64)     36928       conv2d_6[0][0]                   
____________________________________________________________________________________________

In [26]:
# training
optimizer = tf.keras.optimizers.RMSprop()
vae_model.compile(optimizer, loss=reconstruction_loss)
vae_model.fit(train_data, epochs=100, validation_data=test_data, callbacks=[model_tb, model_sdw, model_testd])

Epoch 1/100
   3/3750 [..............................] - ETA: 3:24 - loss: 731.3918   WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0105s vs `on_train_batch_end` time: 0.0147s). Check your callbacks.


3750/3750 [==============================] - 48s 12ms/step - loss: 279.2534 - val_loss: 263.9344


Epoch 2/100
3750/3750 [==============================] - 41s 11ms/step - loss: 261.6925 - val_loss: 261.2047


Epoch 3/100
3750/3750 [==============================] - 39s 10ms/step - loss: 258.7360 - val_loss: 259.2366


Epoch 4/100
3750/3750 [==============================] - 38s 10ms/step - loss: 257.4207 - val_loss: 258.6277


...

Epoch 38/100
3750/3750 [==============================] - 38s 10ms/step - loss: 250.5378 - val_loss: 253.6452
Epoch 39/100
3750/3750 [==============================] - 38s 10ms/step - loss: 250.4943 - val_loss: 254.4371
Epoch 40/100
3750/3750 [==============================] - 38s 10ms/step - loss: 250.4179 - val_loss: 254.2538
Epoch 41/100
3750/3750 [==============================] - 38s 10ms/step - loss: 250.3137 - val_loss: 254.1105
Epoch 42/100
3750/3750 [==============================] - 38s 10ms/step - loss: 250.3222 - val_loss: 252.6876


Epoch 43/100
3750/3750 [==============================] - 38s 10ms/step - loss: 250.2821 - val_loss: 253.4486
Epoch 44/100
3750/3750 [==============================] - 38s 10ms/step - loss: 250.2964 - val_loss: 252.2595


Epoch 45/100
3750/3750 [==============================] - 38s 10ms/step - loss: 250.2781 - val_loss: 252.8003
Epoch 46/100
3750/3750 [==============================] - 38s 10ms/step - loss: 250.1600 - val_loss: 254.5246
Epoch 47/100
3750/3750 [==============================] - 38s 10ms/step - loss: 250.2063 - val_loss: 253.3371
Epoch 48/100
3750/3750 [==============================] - 39s 10ms/step - loss: 250.0836 - val_loss: 252.6102
Epoch 49/100
3750/3750 [==============================] - 38s 10ms/step - loss: 250.1071 - val_loss: 252.5626
Epoch 50/100
3750/3750 [==============================] - 38s 10ms/step - loss: 250.0383 - val_loss: 252.6380
Epoch 51/100
3750/3750 [==============================] - 38s 10ms/step - loss: 250.0123 - val_loss: 253.4164
Epoch 52/100
3750/3750 [==============================] - 38s 10ms/step - loss: 250.0518 - val_loss: 254.3592
Epoch 53/100
3750/3750 [==============================] - 38s 10ms/step - loss: 249.9666 - val_loss: 252.9971
Epoch 54/1

Epoch 69/100
3750/3750 [==============================] - 39s 10ms/step - loss: 249.7007 - val_loss: 253.3717
Epoch 70/100
3750/3750 [==============================] - 38s 10ms/step - loss: 249.7237 - val_loss: 252.7285
Epoch 71/100
3750/3750 [==============================] - 39s 10ms/step - loss: 249.6452 - val_loss: 251.9875


Epoch 72/100
3750/3750 [==============================] - 38s 10ms/step - loss: 249.8496 - val_loss: 254.1579
Epoch 73/100
3750/3750 [==============================] - 39s 10ms/step - loss: 249.6961 - val_loss: 252.6851
Epoch 74/100
3750/3750 [==============================] - 39s 10ms/step - loss: 249.7754 - val_loss: 252.3361
Epoch 75/100
3750/3750 [==============================] - 38s 10ms/step - loss: 249.6754 - val_loss: 253.6768
Epoch 76/100
3750/3750 [==============================] - 43s 11ms/step - loss: 249.6519 - val_loss: 252.4980
Epoch 77/100
3750/3750 [==============================] - 41s 11ms/step - loss: 249.7128 - val_loss: 252.2873
Epoch 78/100
3750/3750 [==============================] - 41s 11ms/step - loss: 249.6005 - val_loss: 253.1813
Epoch 79/100
3750/3750 [==============================] - 43s 11ms/step - loss: 249.6343 - val_loss: 252.5701
Epoch 80/100
3750/3750 [==============================] - 39s 11ms/step - loss: 249.5785 - val_loss: 253.1637
Epoch 81/1